In [21]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import pandas as pd
from keras.models import load_model
import glob
from PIL import Image
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc, f1_score
import scikitplot as skplt
from keras.applications.mobilenetv2 import MobileNetV2, preprocess_input
import matplotlib.pyplot as plt

In [2]:
test = pd.read_csv("test.csv")

In [33]:
size = (224,224)
x_test_ext = []
y_test_ext = []

x_test = []
y_test = []

x_test_middle = []

for row in tqdm(test.iterrows()):
    d = row[1]
    
    image = Image.open("/home/yannis/Developpement/ppd-GAN-for-medical-imaging/dataset-1/data/original/all/" + d["name"])
    img1 = np.array(image.crop((20, 0, 460+20, 460)).resize(size)).astype(np.float32)
    img2 = np.array(image.crop((120, 0, 580, 460)).resize(size)).astype(np.float32)
    img3 = np.array(image.crop((240-20, 0, 700-20, 460)).resize(size)).astype(np.float32)
    
    imgs = [ preprocess_input(img) for img in [img1, img2, img3] ]
    labels = [ int(d["label"]) for x in range(0,3) ]
    
    x_test_ext.extend(imgs)
    y_test_ext.extend(labels)
    
    x_test.append(imgs)
    y_test.append(int(d["label"]))
    
    x_test_middle.append( imgs[1] )

x_test_ext = np.array(x_test_ext)
y_test_ext = np.array(y_test_ext)

x_test = np.array(x_test)
y_test = np.array(y_test)
x_test_middle = np.array(x_test_middle)

1585it [00:18, 84.60it/s]


In [18]:
def mean_score(model, y_test, x_test):
    pred = []
    for x in x_test:
        p2 = model.predict(x)
        pred.append(np.mean(p2, axis=0))
    pred = np.array(pred)
    print(accuracy_score(y_test, np.argmax(pred, axis=1)))

In [18]:
models = glob.glob("../models/dataset-1/Comparaison/*/*")

In [19]:
models
del models[5]
del models[8]

In [20]:
models

['../models/dataset-1/Comparaison/DenseNet121_p3_original_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_p3_original_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_p0_original_128x128/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_only-stylegan_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_only_pgan_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_real-pggan_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_real-pggan20k_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_real-pggan30k_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_only-stylegan-rdm_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_only-stylegan-rdm_224x224/best_model_epoch-84.hdf5',
 '../models/dataset-1/Comparaison/Mobilnet_only-mixgan_224x224/best_model_epoch.hdf5',
 '../models/dataset-1/

In [31]:
for md in models:
    if "224x224" in md:
        print(" ---- ", md.split("/")[-2])
        model = load_model(md, compile=False)
        
        p1 = model.predict(x_test_ext)
        print(accuracy_score(y_test_ext, np.argmax(p1, axis=1)))
        
        mean_score(model, y_test, x_test)

 ----  DenseNet121_p3_original_224x224
0.5667718191377498
0.5659305993690852
 ----  Mobilnet_p3_original_224x224
0.9324921135646688
0.9375394321766561
 ----  Mobilnet_only-stylegan_224x224
0.8426919032597266
0.8485804416403786
 ----  Mobilnet_only_pgan_224x224
0.8937960042060988
0.898422712933754
 ----  Mobilnet_real-pggan_224x224
0.9528916929547845
0.9545741324921135
 ----  Mobilnet_real-pggan20k_224x224
0.9465825446898002
0.9488958990536278
 ----  Mobilnet_real-pggan30k_224x224
0.9186119873817035
0.922397476340694
 ----  Mobilnet_only-stylegan-rdm_224x224
0.8422712933753943
0.8460567823343849
 ----  Mobilnet_only-stylegan-rdm_224x224
0.8365930599369085
0.8485804416403786
 ----  Mobilnet_only-mixgan_224x224
0.9320715036803365
0.9394321766561514
 ----  Mobilnet_only-mixgan_224x224
0.9427970557308096
0.9495268138801262
 ----  Mobilnet_p0_original_224x224
0.919453207150368
0.9287066246056782
 ----  Mobilnet_real-stylegan20K_224x224
0.9575184016824395
0.9583596214511041
 ----  Mobilnet_on

In [33]:
for md in models:
    if "128x128" in md:
        print(" ---- ", md.split("/")[-2])
        model = load_model(md, compile=False)
        
        p1 = model.predict(x_test_ext)
        print(accuracy_score(y_test_ext, np.argmax(p1, axis=1)))
        
        mean_score(model, y_test, x_test)

 ----  Mobilnet_p0_original_128x128
0.903049421661409
0.9078864353312303
 ----  Mobilnet_p3_original_128x128
0.9118822292323869
0.9230283911671924


In [36]:
models = glob.glob("../models/dataset-1/Comparaison_2/*/*")

In [37]:
for md in models:
    print(" ============= ", md.split("/")[-2], " ================")
    model = load_model(md, compile=False)

    p1 = model.predict(x_test_ext)
    print(accuracy_score(y_test_ext, np.argmax(p1, axis=1)))
#     print(" => ", roc_auc_score(y_test_ext, p1[:,1]))
    print(" --------------- ")
    mean_score(model, y_test, x_test)
    print(" --------------- ")
    p2 = model.predict(x_test_middle)
    print(accuracy_score(y_test, np.argmax(p2, axis=1)))
#     print(" => ", roc_auc_score(y_test, p2[:,1]))
    print(" ")

 =============  Mobilnet_real-mixgan_n8  ================
0.9531019978969506
 --------------- 
0.955205047318612
 --------------- 
0.9533123028391167
 
 =============  Mobilnet_real-stylegan_n6  ================
0.9404837013669821
 --------------- 
0.9413249211356467
 --------------- 
0.9425867507886435
 
 =============  Mobilnet_real-mixgan_n8_3  ================
0.9602523659305994
 --------------- 
0.9615141955835962
 --------------- 
0.9589905362776026
 
 =============  Mobilnet_real-mixgan_n8_3  ================
0.9556256572029442
 --------------- 
0.9596214511041009
 --------------- 
0.9533123028391167
 
 =============  Mobilnet_real-pggan_n6  ================
0.9364879074658254
 --------------- 
0.9375394321766561
 --------------- 
0.934384858044164
 


In [5]:
models = glob.glob("../models/dataset-1/Comparaison_2/*_3/*")

In [35]:
for md in models:
    print(" ============= ", md.split("/")[-2], " ================")
    model = load_model(md, compile=False)

    p1 = model.predict(x_test_ext)
    print(accuracy_score(y_test_ext, np.argmax(p1, axis=1)))
#     print(" => ", roc_auc_score(y_test_ext, p1[:,1]))
    print(" --------------- ")
    mean_score(model, y_test, x_test)
    print(" --------------- ")
    p2 = model.predict(x_test_middle)
    print(accuracy_score(y_test, np.argmax(p2, axis=1)))
#     print(" => ", roc_auc_score(y_test, p2[:,1]))
    print(" ")

 =============  Mobilnet_real-mixgan_n8_3  ================
0.9602523659305994
 --------------- 
0.9615141955835962
 --------------- 
0.9589905362776026
 
 =============  Mobilnet_real-mixgan_n8_3  ================
0.9556256572029442
 --------------- 
0.9596214511041009
 --------------- 
0.9533123028391167
 


### Resultat comparaison data

In [38]:
models = glob.glob("../models/dataset-1/Comparaison-Size/*/*")

In [41]:
for md in models:
    print(" ============= ", md.split("/")[-2], " ================")
    model = load_model(md, compile=False)
    p1 = model.predict(x_test_ext)
    print(accuracy_score(y_test_ext, np.argmax(p1, axis=1)))
    mean_score(model, y_test, x_test)
    p2 = model.predict(x_test_middle)
    print(accuracy_score(y_test, np.argmax(p2, axis=1)))
    print(" ")

 =============  stylegan-7k  ================
0.8292323869610936
0.8315457413249211
0.8309148264984227
 
 =============  stylegan-20k  ================
0.8128286014721346
0.8145110410094637
0.8082018927444795
 
 =============  stylegan-30k  ================
0.8378548895899054
0.8454258675078864
0.8435331230283911
 
 =============  pgan-7k  ================
0.9116719242902208
0.9154574132492114
0.9066246056782334
 
 =============  mixgan-30k  ================
0.9449001051524711
0.9463722397476341
0.9438485804416403
 
 =============  mixgan-20k  ================
0.9341745531019979
0.9451104100946373
0.9337539432176656
 
 =============  mixgan-7k  ================
0.943217665615142
0.9457413249211356
0.943217665615142
 
 =============  pgan-12k  ================
0.9137749737118822
0.9173501577287066
0.9154574132492114
 
 =============  mixgan-12k  ================
0.934384858044164
0.9369085173501577
0.9337539432176656
 
 =============  stylegan-12k  ================
0.8056782334384858
0.

## Mobilnet_only_pgan_224x224

In [26]:
model = load_model("../models/dataset-1/Comparaison/Mobilnet_p3_original_224x224/best_model_epoch.hdf5")
p1 = model.predict(x_test_ext)
accuracy_score(y_test_ext, np.argmax(p1, axis=1))

0.9324921135646688

# Mobilnet_p3_original_224x224

In [23]:
model = load_model("../models/dataset-1/Comparaison/Mobilnet_p3_original_224x224/best_model_epoch.hdf5")

In [24]:
p1 = model.predict(x_test_ext)
accuracy_score(y_test_ext, np.argmax(p1, axis=1))

0.9324921135646688

In [25]:
pred = []
for x in x_test:
    p2 = model.predict(x)
    pred.append(np.mean(p2, axis=0))
pred = np.array(pred)
accuracy_score(y_test, np.argmax(pred, axis=1))

0.9375394321766561

# Mobilnet_p3_norm_224x224

In [3]:
model = load_model("../models/dataset-1/Comparaison/Mobilnet_p3_norm_224x224/best_model_epoch.hdf5")

In [13]:
p1 = model.predict(x_test_ext)
accuracy_score(y_test_ext, np.argmax(p1, axis=1))

0.9062039957939012

In [21]:
pred = []
for x in x_test:
    p2 = model.predict(x)
    pred.append(np.mean(p2, axis=0))
pred = np.array(pred)
accuracy_score(y_test, np.argmax(pred, axis=1))

0.9129337539432176

## 8 classes 

## CNN 

In [42]:
model = load_model("../models/dataset-1/Comparaison-Cat/Mobilnet_p3/best_model_epoch.hdf5")

In [47]:
from collections import defaultdict
test_p3 = pd.read_csv("test_p3.csv")
x_test = defaultdict(list)
y_test = defaultdict(list)
for row in tqdm(test_p3.iterrows()):
    d = row[1]
    
    image = Image.open("/home/yannis/Developpement/ppd-GAN-for-medical-imaging/dataset-1/data/original/p3/" + d["name"])
    img = np.array(image).astype(np.float32)
    
    x_test[d["name"][3:]].append(preprocess_input(img))
    y_test[d["name"][3:]].append(int(d["label_cat"]))
# x_test = np.array(x_test)
# y_test = np.array(y_test)

4755it [00:10, 451.73it/s]


In [48]:
pred = []
ytrue = []
for k, v in tqdm(x_test.items()):
    p2 = model.predict(np.array(v))
    pred.append(np.mean(p2, axis=0))
    ytrue.append(y_test[k][0])
pred = np.array(pred)
print(accuracy_score(ytrue, np.argmax(pred, axis=1)))

100%|██████████| 1585/1585 [00:38<00:00, 41.23it/s]

0.5160883280757098


In [58]:
pred = []
ytrue = []
for k, v in tqdm(x_test.items()):
    pred.extend(v)
    ytrue.extend(y_test[k])
pred = np.array(pred)

p1 = model.predict(pred)
accuracy_score(ytrue, np.argmax(p1, axis=1))

100%|██████████| 1585/1585 [00:00<00:00, 901053.38it/s]


0.5125131440588854

### ACGAN 

In [18]:
from collections import defaultdict

In [27]:
test_p3 = pd.read_csv("test_p3.csv")
x_test = defaultdict(list)
y_test = defaultdict(list)
for row in tqdm(test_p3.iterrows()):
    d = row[1]
    
    image = Image.open("/home/yannis/Developpement/ppd-GAN-for-medical-imaging/dataset-1/data/original/p3/" + d["name"])
    img = np.array(image).astype(np.float32)
    
    x_test[d["name"][3:]].append(preprocess_input(img))
    y_test[d["name"][3:]].append(int(d["label_cat"]))

4755it [00:13, 354.88it/s]


In [16]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [12]:
model = load_model("../models/dataset-1/Comparaison-Cat/Mobilnet_ACGAN/model_epoch-16.hdf5")

In [14]:
p1 = model.predict(x_test)
accuracy_score(y_test, np.argmax(p1, axis=1))

0.5514195583596214

In [17]:
p1 = model.predict(x_test)
accuracy_score(y_test, np.argmax(p1, axis=1))

0.5526813880126183

In [36]:
pred = []
ytrue = []
for k, v in tqdm(x_test.items()):
    p2 = model.predict(np.array(v))
    pred.append(np.mean(p2, axis=0))
    ytrue.append(y_test[k][0])
pred = np.array(pred)
print(accuracy_score(ytrue, np.argmax(pred, axis=1)))

100%|██████████| 1585/1585 [02:22<00:00, 11.45it/s]

0.5545741324921135
